Ładujemy pakiety

In [3]:
import pandas as pd 
import librosa

import numpy as np
import os
import torch
import time
#from google.colab import drive
from numpy import save

Wczytujemy dane

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
statusy = pd.read_csv('/content/drive/My Drive/Projekt_ptaki/statusy.csv', encoding='cp1250')

In [ ]:
statusy.head()

Rozpakowujemy plik z dźwiękiem - jednorazowo, jeżeli nie jest rozpakowany na drive

In [ ]:
#!pip install pyunpack
#!pip install patool
#from pyunpack import Archive
#Archive('/content/drive/My Drive/Projekt_ptaki/01_Dzwieki.zip').extractall('/content/drive/My Drive/Projekt_ptaki/01_Dzwieki')

Właściwe wczytywanie dźwięku i tworzenie charakterystyk

In [ ]:
def extract_features(files):
  try:
      file_name = os.path.join(os.path.abspath('/content/drive/My Drive/Projekt_ptaki/01_Dzwieki/01_Dzwieki')+'/'+str(files.link))
      X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
      stft = np.abs(librosa.stft(X))
      chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
      mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
      contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
      tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
      sr=sample_rate).T,axis=0)
  
  except ZeroDivisionError:
      mfccs = np.zeros(40)
      chroma = np.zeros(12)
      mel = np.zeros(128)
      contrast = np.zeros(7)
      tonnetz = np.zeros(6)

  return mfccs, chroma, mel, contrast, tonnetz

In [ ]:
import timeit
from tqdm import tqdm

tqdm.pandas()

start = timeit.default_timer()

train_features = statusy.progress_apply(extract_features, axis=1)

stop = timeit.default_timer()

print('Time: ', stop - start)  

features_train = []
for i in range(0, len(train_features)):
    features_train.append(np.concatenate((
        train_features[i][0],
        train_features[i][1], 
        train_features[i][2], 
        train_features[i][3],
        train_features[i][4]), axis=0))
    
X_train = np.array(features_train)

np.save("drive/My Drive/Projekt_ptaki/cechy.npy", X_train)


 13%|█▎        | 326/2585 [34:31<3:51:32,  6.15s/it]/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
 23%|██▎       | 591/2585 [1:04:03<2:42:16,  4.88s/it]/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
 57%|█████▋    | 1479/2585 [2:17:11<1:30:49,  4.93s/it]/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
 66%|██████▌   | 1709/2585 [2:38:17<1:47:56,  7.39s/it]/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from

Time:  15442.091955988


In [ ]:
X_train = np.load("drive/My Drive/Projekt_ptaki/cechy.npy")